In [2]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
# Initialize sparqlWrapper
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

In [4]:
# Set paths
path_dbpedia='data/dbpedia/'
path_movielens='data/movielens/'

In [5]:
# Load files
movies_dbpedia_df=pd.read_csv(path_dbpedia+'MappingMovielens2DBpedia-1.2.tsv', sep='\t', header=None, names=['movieId','title','dbpedia_link'])
movies_movielens_df=pd.read_csv(path_movielens+'movies.csv')
# Merge dbpedia file with the one from movielens
movies_df = pd.merge(movies_dbpedia_df, movies_movielens_df[['movieId','genres']], how='inner', on='movieId')

In [30]:
# Define query functions
def query_dbpedia_single(query):
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    sparql_results = sparql.query().convert()
    results = sparql_results["results"]["bindings"]
    if results != []:
        uri = ''
        label = ''
        if "uri" in results[0]:
            uri = results[0]["uri"]["value"]
        if "label" in results[0]:
            label = results[0]["label"]["value"]
        return uri, label
    else:
        return '', ''

def query_dbpedia_multiple(query):
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    array = []
    for result in results["results"]["bindings"]:
        array.append(result["label"]["value"])

    values = '|'.join(array)
    return values

In [9]:
movie = movies_df.iloc[8, :]
movie_id = movie.get('movie_id')
movie_title = movie.get('movie_title')
movie_dbpedia = movie.get('dbpedia_link')
movie

movieId                                                      3719
title                                 Love's Labour's Lost (2000)
dbpedia_link    http://dbpedia.org/resource/Love's_Labour's_Lo...
genres                                             Comedy|Romance
Name: 8, dtype: object

In [12]:
movie_dbpedia

"http://dbpedia.org/resource/Love's_Labour's_Lost_(2000_film)"

In [10]:
# Set SPARQL queries
#sparql_query_single_value = "SELECT ?director ?country (MAX(?runtime) AS ?runtime) ?writer WHERE {{ ?dir rdfs:label ?director .?coun rdfs:label ?country .?wr rdfs:label ?writer . <{0}> dbo:writer ?wr . <{0}> dbo:runtime ?runtime . <{0}> dbo:country ?coun . <{0}> dbo:director ?dir . FILTER (lang(?director) = 'en' AND lang(?country) = 'en' AND lang(?writer) = 'en')}}".format(movie_dbpedia)
#sparql_query_country_failover = "SELECT ?label WHERE { <%s> dbp:country ?label}" % movie_dbpedia
sparql_query_director = "SELECT ?uri ?label WHERE { ?uri rdfs:label ?label . <%s> dbo:director ?uri FILTER (lang(?label) = 'en')}" % movie_dbpedia
sparql_query_country = "SELECT ?uri ?label WHERE { ?uri rdfs:label ?label . <%s> dbo:country ?uri FILTER (lang(?label) = 'en')}" % movie_dbpedia
sparql_query_runtime = "SELECT ('' AS ?uri), (MAX(?runtime) AS ?label) WHERE { <%s> dbo:runtime ?runtime }" % movie_dbpedia
sparql_query_writer = "SELECT ?uri ?label WHERE { ?uri rdfs:label ?label . <%s> dbo:writer ?uri FILTER (lang(?label) = 'en')}" % movie_dbpedia
sparql_query_starring = "SELECT ?starring ?label WHERE { ?starring rdfs:label ?label . <%s> dbo:starring ?starring FILTER (lang(?label) = 'en')}" % movie_dbpedia
sparql_query_languages = "SELECT ?language ?label WHERE { ?language rdfs:label ?label . <%s> dbo:language ?language FILTER (lang(?label) = 'en')}" % movie_dbpedia
sparql_query_studios = "SELECT ?studios ?label WHERE { ?studios rdfs:label ?label . <%s> dbp:studio ?studios FILTER (lang(?label) = 'en')}" % movie_dbpedia

In [31]:
director_uri, director_name = query_dbpedia_single(sparql_query_director)
country_uri, country_name = query_dbpedia_single(sparql_query_country)
runtime_uri, runtime = query_dbpedia_single(sparql_query_runtime)
writer_uri, writer_name = query_dbpedia_single(sparql_query_writer)
starring = query_dbpedia_multiple(sparql_query_starring)
languages = query_dbpedia_multiple(sparql_query_languages)
studios = query_dbpedia_multiple(sparql_query_studios)

In [32]:
movie['director_uri'] = director_uri
movie['director'] = director_name
movie['country_uri'] = country_uri
movie['country'] = country_name
movie['runtime'] = runtime
movie['writer_uri'] = writer_uri
movie['writer'] = writer_name
movie['starring'] = starring
movie['languagues'] = languages
movie['studios'] = studios

print(movie)

movieId                                                      3719
title                                 Love's Labour's Lost (2000)
dbpedia_link    http://dbpedia.org/resource/Love's_Labour's_Lo...
genres                                             Comedy|Romance
director_uri                                                     
director                                                         
country_uri                                                      
country                                                          
runtime                                                          
writer_uri                                                       
writer                                                           
starring                                                         
languagues                                                       
studios                                                          
Name: 8, dtype: object


# Now everything together

In [8]:
movies_enriched_df = pd.DataFrame(columns=['movieId', 'title', 'dbpedia_link','genres','director_uri','director','country_uri','country','runtime','writer_uri','writer','starring', 'languagues', 'studios'])

for i in range(len(movies_df)) : 
    try:
        if i % 5 == 0:
            print(i)
        movie = movies_df.iloc[i, :]
        movie_id = movie.get('movie_id')
        movie_title = movie.get('movie_title')
        movie_dbpedia = movie.get('dbpedia_link')

        # Set SPARQL queries
        sparql_query_director = "SELECT ?uri ?label WHERE { ?uri rdfs:label ?label . <%s> dbo:director ?uri FILTER (lang(?label) = 'en')}" % movie_dbpedia
        sparql_query_country = "SELECT ?uri ?label WHERE { ?uri rdfs:label ?label . <%s> dbo:country ?uri FILTER (lang(?label) = 'en')}" % movie_dbpedia
        sparql_query_runtime = "SELECT ('' AS ?uri), (MAX(?runtime) AS ?label) WHERE { <%s> dbo:runtime ?runtime }" % movie_dbpedia
        sparql_query_writer = "SELECT ?uri ?label WHERE { ?uri rdfs:label ?label . <%s> dbo:writer ?uri FILTER (lang(?label) = 'en')}" % movie_dbpedia
        sparql_query_starring = "SELECT ?starring ?label WHERE { ?starring rdfs:label ?label . <%s> dbo:starring ?starring FILTER (lang(?label) = 'en')}" % movie_dbpedia
        sparql_query_languages = "SELECT ?language ?label WHERE { ?language rdfs:label ?label . <%s> dbo:language ?language FILTER (lang(?label) = 'en')}" % movie_dbpedia
        sparql_query_studios = "SELECT ?studios ?label WHERE { ?studios rdfs:label ?label . <%s> dbp:studio ?studios FILTER (lang(?label) = 'en')}" % movie_dbpedia

        # Query DBPedia
        director_uri, director_name = query_dbpedia_single(sparql_query_director)
        country_uri, country_name = query_dbpedia_single(sparql_query_country)
        runtime_uri, runtime = query_dbpedia_single(sparql_query_runtime)
        writer_uri, writer_name = query_dbpedia_single(sparql_query_writer)
        starring = query_dbpedia_multiple(sparql_query_starring)
        languages = query_dbpedia_multiple(sparql_query_languages)
        studios = query_dbpedia_multiple(sparql_query_studios)

        movie['director_uri'] = director_uri
        movie['director'] = director_name
        movie['country_uri'] = country_uri
        movie['country'] = country_name
        movie['runtime'] = runtime
        movie['writer_uri'] = writer_uri
        movie['writer'] = writer_name
        movie['starring'] = starring
        movie['languagues'] = languages
        movie['studios'] = studios

        movies_enriched_df = movies_enriched_df.append(movie)    
    except:
        print("An exception occurred at index:", i)
        break
    
movies_enriched_df.to_csv('data/processed/movies_enriched.csv')


0
5
An exception occurred at index: 8
